In [3]:
profile_matrix = {
    "Billing": "1-0:98.1.0*255",
    "Daily": "1-0:99.2.0*255",
    "BlockLoad": "1-0:99.1.0*255",
    "Instantaneous": "1-0:94.91.0*255"
}

while True:
    print("For Billing Profile Write 'Billing' and hit enter")
    print("For Daily Profile Write 'Daily' and hit enter")
    print("For BlockLoad Profile Write 'BlockLoad' and hit enter")
    print("For Instantaneous Profile Write 'Instantaneous' and hit enter")
    
    prof = input("Enter the profile you want to get data for: ")
    
    profile = profile_matrix.get(prof)
    
    if profile:
        print(f"Profile selected: {prof}, Code: {profile}")
        break  # Exit the loop if valid input is provided
    else:
        print("Invalid profile, please try again.\n")

For Billing Profile Write 'Billing' and hit enter
For Daily Profile Write 'Daily' and hit enter
For BlockLoad Profile Write 'BlockLoad' and hit enter
For Instantaneous Profile Write 'Instantaneous' and hit enter


Enter the profile you want to get data for:  avnish


Invalid profile, please try again.

For Billing Profile Write 'Billing' and hit enter
For Daily Profile Write 'Daily' and hit enter
For BlockLoad Profile Write 'BlockLoad' and hit enter
For Instantaneous Profile Write 'Instantaneous' and hit enter


Enter the profile you want to get data for:  Daily


Profile selected: Daily, Code: 1-0:99.2.0*255
